#Возможно ли иметь выигрыш в долгосрочной перспективе в такой сфере, как ставки на теннисные матчи? Для ответа на данный вопрос я проверю каким будет выигрыш/проигрыш игрока что в асболютных, что в относительных значениях при использовании некоторых ставочных стратегий

#Загрузка необходимых для работы библиотек и данных (данные можно найти по ссылке:https://www.kaggle.com/datasets/edoardoba/atp-tennis-data)

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
import lightgbm as ltb
import copy

In [2]:
data = pd.read_csv('tennis_data.csv', parse_dates=['Date'])

<ipython-input-2-bbe615a3abf0>:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('tennis_data.csv', parse_dates=['Date'])


#Предобработка данных

Для проверки стратегий я буду использовать матчи серий ATP 250, ATP 500, Masters 1000 и турниров Большого Шлема, которые закончились победой одного из игороков по итогам матча (то есть матчи, где были снятия по состоянию здоровья, из-за дисквалификации и тд не будут использоваться). Также не будут использоваться матчи, где нет фаворита по мнению букмекера.

In [3]:
data = data[(data['Series']=='ATP250') | (data['Series']=='ATP500') | (data['Series']=='Masters 1000') | (data['Series']=='Grand Slam')]
data= data[(data['Comment']=='Completed') & (data['Surface']!='Carpet')].reset_index(drop=True)
data = data.dropna(subset=['B365W', 'B365L']).reset_index(drop=True)
data = data.drop(data[(data['B365W'])==(data['B365L'])].index).reset_index(drop=True)

In [4]:
data = data.drop_duplicates().reset_index(drop=True)

также удалим те даннные, в которых может быть ошибка, коэффициент на победу фаворита меньше 1, а также те матчи, в котрых победитель взял меньше партий, чем проигравший.

In [5]:
data = data.drop(data[(data['Wsets'])<(data['Lsets'])].index).reset_index(drop=True)
data = data.drop(data[(data['B365W']<=1) | (data['B365L']<=1)].index).reset_index(drop=True)

#Стратегия флэта

Стратегия флэта заключается в том, что игрок ставить одинаковую сумму на каждую игру независимо от коэффициентов на матч. Стоит отметить, что будут использоваться наивные предположения: только фаворит игры побеждает или только аутсайдер побеждает. Фаворит - тот игрок, на которого букмекерский коэффициент ниже.

В качестве метрик качества предсказаний будут чистый выигрыш/проигыш игрока, а также ROI = выигрыш/сумма ставок

флэт ставки на фаворита. Стратегия заключается в том, что игрок ставит на того игрока, на кого коэффициент перед матчем меньше (то есть вероятность его победы оценивается выше, чем вероятность поражения).

In [6]:
def flat_fav(data):
  data1 = copy.deepcopy(data)
  data1['fav_coef'] = data1[['B365W', 'B365L']].min(axis=1)
  data1['fav_win'] = (data1['fav_coef']==data1['B365W'])*1
  data1['final'] = data1['fav_coef']*data1['fav_win']-1
  profit=round(data1['final'].sum(),2)
  ROI = round(profit/len(data1)*100,2)
  return f'На дистанции {len(data1)} ставок выигрыш составил {profit} единиц, ROI составил {ROI}%'

In [7]:
flat_fav(data)

'На дистанции 31587 ставок выигрыш составил -1608.86 единиц, ROI составил -5.09%'

флэт ставки на аутсайдера. Стратегия заключается в том, что игрок ставит на того игрока, на кого коэффициент перед матчем выше (то есть вероятность его победы оценивается ниже, чем вероятность поражения).

In [8]:
def flat_under(data):
  data1 = copy.deepcopy(data)
  data1['under_coef'] = data1[['B365W', 'B365L']].max(axis=1)
  data1['under_win'] = (data1['under_coef']==data1['B365W'])*1
  data1['final'] = data1['under_coef']*data1['under_win']-1
  profit=round(data1['final'].sum(),2)
  ROI = round(profit/len(data1)*100,2)
  return f'На дистанции {len(data1)} ставок выигрыш составил {profit} единиц, ROI составил {ROI}%'

In [9]:
flat_under(data)

'На дистанции 31587 ставок выигрыш составил -4006.47 единиц, ROI составил -12.68%'

#Стратегия в зависимости от коэффициента на фаворита (чем ниже коэффициент, тем выше сумма ставки)

стратегия в ставках на фаворита

In [10]:
def non_flat_fav(data):
  data1 = copy.deepcopy(data)
  data1['fav_coef'] = data1[['B365W', 'B365L']].min(axis=1)
  data1['fav_win'] = (data1['fav_coef']==data1['B365W'])*1
  data1['unit'] = 1+1/data1['fav_coef']
  data1['final'] = (data1['fav_coef']*data1['fav_win'])*data1['unit']-data1['unit']
  profit=round(data1['final'].sum(),2)
  betted = round(data1['unit'].sum())
  ROI = round(profit/betted*100,2)
  return f'На дистанции {len(data1)} ставок выигрыш  составил {profit} единиц, ROI составил {ROI}%'

In [11]:
non_flat_fav(data)

'На дистанции 31587 ставок выигрыш  составил -2724.29 единиц, ROI составил -4.95%'

стратегия в ставках на аутсайдера

In [12]:
def non_flat_under(data):
  data1 = copy.deepcopy(data)
  data1['fav_coef'] = data1[['B365W', 'B365L']].min(axis=1)
  data1['under_coef'] = data1[['B365W', 'B365L']].max(axis=1)
  data1['under_win'] = (data1['under_coef']==data1['B365W'])*1
  data1['unit'] = 2-1/data1['fav_coef']
  data1['final'] = (data1['under_coef']*data1['under_win'])*data1['unit']-data1['unit']
  profit=round(data1['final'].sum(),2)
  betted = round(data1['unit'].sum())
  ROI = round(profit/betted*100,2)
  return f'На дистанции {len(data1)} ставок выигрыш  составил {profit} единиц, ROI составил {ROI}%'

In [13]:
non_flat_under(data)

'На дистанции 31587 ставок выигрыш  составил -4718.05 единиц, ROI составил -11.88%'

#Стратегия не ставить на некоторые матчи в зависимости от коэффицента

Стратегия заключается в том, что ставим в том случае, если коэффициент ниже определенного значения

ставки на фаворита по этой стратегии

In [14]:
def non_flat_fav3(data,coef):
  data1 = copy.deepcopy(data)
  data1['fav_coef'] = data1[['B365W', 'B365L']].min(axis=1)
  data1['fav_win'] = (data1['fav_coef']==data1['B365W'])*1
  data1['unit'] = (data1['fav_coef']<coef)*1
  data1['final'] = (data1['fav_coef']*data1['fav_win'])*data1['unit']-data1['unit']
  profit=round(data1['final'].sum(),2)
  betted = round(data1['unit'].sum())
  ROI = round(profit/betted*100,2)
  return f'На дистанции {betted} ставок выигрыш  составил {profit} единиц, ROI составил {ROI}%'

In [15]:
coef_var = np.arange(1.0, 2.0,0.1)
coef_var1 = np.arange(2, 3.5,0.2)

In [16]:
for i in range(len(coef_var)):
  print(f'при ставках на коэф меньше {round(coef_var[i],2)} ', non_flat_fav3(data, coef_var[i]))

при ставках на коэф меньше 1.0  На дистанции 0 ставок выигрыш  составил 0.0 единиц, ROI составил nan%
при ставках на коэф меньше 1.1  На дистанции 3168 ставок выигрыш  составил -65.2 единиц, ROI составил -2.06%
при ставках на коэф меньше 1.2  На дистанции 8361 ставок выигрыш  составил -249.23 единиц, ROI составил -2.98%
при ставках на коэф меньше 1.3  На дистанции 12845 ставок выигрыш  составил -327.01 единиц, ROI составил -2.55%
при ставках на коэф меньше 1.4  На дистанции 18196 ставок выигрыш  составил -571.37 единиц, ROI составил -3.14%
при ставках на коэф меньше 1.5  На дистанции 21725 ставок выигрыш  составил -821.51 единиц, ROI составил -3.78%
при ставках на коэф меньше 1.6  На дистанции 25354 ставок выигрыш  составил -1101.0 единиц, ROI составил -4.34%
при ставках на коэф меньше 1.7  На дистанции 28553 ставок выигрыш  составил -1363.07 единиц, ROI составил -4.77%
при ставках на коэф меньше 1.8  На дистанции 31580 ставок выигрыш  составил -1612.86 единиц, ROI составил -5.11%
при 

<ipython-input-14-d2ddda862dcf>:9: RuntimeWarning: invalid value encountered in double_scalars
  ROI = round(profit/betted*100,2)


ставки на аутсайдера при данной стратегии

In [17]:
def non_flat_under3(data, coef):
  data1 = copy.deepcopy(data)
  data1['under_coef'] = data1[['B365W', 'B365L']].max(axis=1)
  data1['under_win'] = (data1['under_coef']==data1['B365W'])*1
  data1['unit'] = (data1['under_coef']<coef)*1
  data1['final'] = (data1['under_coef']*data1['under_win'])*data1['unit']-data1['unit']
  profit=round(data1['final'].sum(),2)
  betted = round(data1['unit'].sum())
  ROI = round(profit/betted*100,2)
  return f'На дистанции {betted} ставок выигрыш составил {profit} единиц, ROI составил {ROI}%'

In [18]:
for i in range(len(coef_var1)):
  print(f'при ставках на коэф меньше {round(coef_var1[i],2)}', non_flat_under3(data, coef_var1[i]))

при ставках на коэф меньше 2.0 На дистанции 688 ставок выигрыш составил -66.3 единиц, ROI составил -9.64%
при ставках на коэф меньше 2.2 На дистанции 4423 ставок выигрыш составил -256.35 единиц, ROI составил -5.8%
при ставках на коэф меньше 2.4 На дистанции 9469 ставок выигрыш составил -512.62 единиц, ROI составил -5.41%
при ставках на коэф меньше 2.6 На дистанции 11075 ставок выигрыш составил -596.12 единиц, ROI составил -5.38%
при ставках на коэф меньше 2.8 На дистанции 14988 ставок выигрыш составил -857.57 единиц, ROI составил -5.72%
при ставках на коэф меньше 3.0 На дистанции 17147 ставок выигрыш составил -1127.06 единиц, ROI составил -6.57%
при ставках на коэф меньше 3.2 На дистанции 17362 ставок выигрыш составил -1146.86 единиц, ROI составил -6.61%
при ставках на коэф меньше 3.4 На дистанции 19343 ставок выигрыш составил -1327.77 единиц, ROI составил -6.86%


#Ни одна из стратегий не принесла дохода в случае подхода ставок только на фаворита или андердога. Для того, чтобы все-таки придумать выигрышную стратегию, необходимо построить классификатор, который предсказавывает победителя матча и уже с помощью итогов этого классификатора, можно пробовать составлять новые стратегии.

теперь для классификатора добавим такие показатели как результат выступлений в прошлом перед игрой годом и результат выступлений на покрытии (в зависимости от того, на каком покрытии проходит игра) в прошлом перед игрой годом 

In [19]:
data_help = copy.deepcopy(data)

In [20]:
data_help['Year'] = pd.DatetimeIndex(data_help['Date']).year

In [21]:
a = pd.DataFrame(data_help.groupby(['Winner', 'Year']).count())
b = pd.DataFrame(data_help.groupby(['Loser', 'Year']).count())
c = pd.DataFrame(data_help.groupby(['Winner', 'Year', 'Surface']).count())
d = pd.DataFrame(data_help.groupby(['Loser', 'Year', 'Surface']).count())

In [22]:
Wins= a.reset_index().iloc[:,:3]
Losses = b.reset_index().iloc[:,:3]
Wins_surf = c.reset_index().iloc[:,:4]
Losses_surf = d.reset_index().iloc[:,:4]

In [23]:
Wins = Wins.rename(columns={'Winner': 'Player', 'ATP': 'Wins'})
Losses = Losses.rename(columns={'Loser': 'Player', 'ATP': 'Losses'})
Wins_surf = Wins_surf.rename(columns={'Winner': 'Player', 'ATP': 'Wins_surf'})
Losses_surf = Losses_surf.rename(columns={'Loser': 'Player', 'ATP': 'Losses_surf'})

In [24]:
stats_by_year = Wins.merge(Losses, how='right', on = ['Player', 'Year']).fillna(0)
stats_by_year_surf = Wins_surf.merge(Losses_surf, how='right', on = ['Player', 'Year', 'Surface']).fillna(0)

In [25]:
data['prev_year']= pd.DatetimeIndex(data['Date']).year-1
data = data[data['prev_year']>2007].reset_index(drop=True)

In [26]:
a = pd.merge(data, stats_by_year, left_on=  ['Winner', 'prev_year'],
                   right_on= ['Player', 'Year'], 
                   how = 'left')

In [27]:
data = pd.merge(a, stats_by_year, left_on=  ['Loser', 'Year'],
                   right_on= ['Player', 'Year'], 
                   how = 'left')

In [28]:
data = pd.merge(data, stats_by_year_surf, left_on=  ['Winner', 'Year', 'Surface'],
                   right_on= ['Player', 'Year', 'Surface'], 
                   how = 'left')

In [29]:
data = pd.merge(data, stats_by_year_surf, left_on=  ['Loser', 'Year', 'Surface'],
                   right_on= ['Player', 'Year', 'Surface'], 
                   how = 'left')

<ipython-input-29-eedf23de1ad1>:1: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Player_x'} in the result is deprecated and will raise a MergeError in a future version.
  data = pd.merge(data, stats_by_year_surf, left_on=  ['Loser', 'Year', 'Surface'],


добавляем коэффициент на фаворита в матче, а также вспомогательную переменую победа фаворита в матче

In [30]:
data['fav_coef'] = data[['B365W', 'B365L']].min(axis=1)
data['under_coef'] = data[['B365W', 'B365L']].max(axis=1)
data['fav_win'] = (data['fav_coef']==data['B365W'])*1
data['under_win'] = (data['under_coef']==data['B365W'])*1

теперь сравнение игроков по результатам прошлого сезона

In [31]:
data['p1_win_rate'] = data['Wins_x'] - data['Losses_x']
data['p2_win_rate'] = data['Wins_y'] - data['Losses_y']
data['p1_surf_win_rate'] = data['Wins_surf_x'] - data['Losses_surf_x']
data['p2_surf_win_rate'] = data['Wins_surf_y'] - data['Losses_surf_y']

Теперь составляем датасет, в котором параметры считаются уже не относительно победителя и програвшего игрока, а относительно того, кто был фаворитом перед игрой по мнению букмекера.

In [32]:
def fav_attributes(data, column, column1):
  name= 'fav_'+column[1:]
  data[name] = np.where(data['fav_win']>data['under_win'],
                        data[column].combine_first(data[column1]),
                        data[column1].combine_first(data[column]))
  return data

In [33]:
def under_attributes(data, column, column1):
  name= 'under_'+column[1:]
  data[name] = np.where(data['fav_win']<data['under_win'],
                        data[column].combine_first(data[column1]),
                        data[column1].combine_first(data[column]))
  return data

In [34]:
fav_attributes(data,'WRank','LRank')
fav_attributes(data,'WPts','LPts')
fav_attributes(data,'pl1_year_pro','pl2_year_pro')
fav_attributes(data,'pl1_hand','pl2_hand')
fav_attributes(data,'pl1_height','pl2_height')
fav_attributes(data,'p1_win_rate','p2_win_rate')
fav_attributes(data,'p1_surf_win_rate','p2_win_rate')
under_attributes(data,'WRank','LRank')
under_attributes(data,'WPts','LPts')
under_attributes(data,'pl1_year_pro','pl2_year_pro')
under_attributes(data,'pl1_hand','pl2_hand')
under_attributes(data,'pl1_height','pl2_height')
under_attributes(data,'p1_win_rate','p2_win_rate')
under_attributes(data,'p1_surf_win_rate','p2_win_rate')

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner,...,fav_l1_height,fav_1_win_rate,fav_1_surf_win_rate,under_Rank,under_Pts,under_l1_year_pro,under_l1_hand,under_l1_height,under_1_win_rate,under_1_surf_win_rate
0,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Kwon S.W.,...,180.0,3.0,2.0,81.0,823.0,2014.0,Left-Handed,170.0,-7.0,-7.0
1,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Monteiro T.,...,188.0,-2.0,-2.0,89.0,805.0,2011.0,Left-Handed,183.0,-2.0,2.0
2,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Djere L.,...,185.0,-2.0,-4.0,79.0,837.0,2011.0,Right-Handed,180.0,-6.0,-6.0
3,1,Adelaide,Adelaide International 1,2022-01-03,ATP250,Outdoor,Hard,1st Round,3,Johnson S.,...,188.0,-2.0,-2.0,85.0,812.0,2012.0,Right-Handed,188.0,-1.0,0.0
4,1,Adelaide,Adelaide International 1,2022-01-04,ATP250,Outdoor,Hard,1st Round,3,Moutet C.,...,188.0,-5.0,-5.0,92.0,797.0,2016.0,Left-Handed,175.0,-1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31105,64,Paris,BNP Paribas Masters,2009-11-13,Masters 1000,Indoor,Hard,Quarterfinals,3,Nadal R.,...,188.0,6.0,6.0,2.0,8845.0,2001.0,Left-Handed,185.0,21.0,7.0
31106,64,Paris,BNP Paribas Masters,2009-11-13,Masters 1000,Indoor,Hard,Quarterfinals,3,Monfils G.,...,198.0,4.0,4.0,16.0,2055.0,2004.0,Right-Handed,193.0,6.0,2.0
31107,64,Paris,BNP Paribas Masters,2009-11-14,Masters 1000,Indoor,Hard,Semifinals,3,Djokovic N.,...,188.0,15.0,11.0,2.0,8845.0,2001.0,Left-Handed,185.0,21.0,21.0
31108,64,Paris,BNP Paribas Masters,2009-11-14,Masters 1000,Indoor,Hard,Semifinals,3,Monfils G.,...,193.0,6.0,2.0,14.0,2310.0,1996.0,Right-Handed,185.0,3.0,3.0


выбираем необходимые для модели параметры: уровень турнира, покрытие, стадия турнира, котировки на победу игроков, их рейтинг, выступления в прошлом году в целом и покрытии, их рост, опыт, а также исход матча (победа фаворита или нет).

In [35]:
model_data=data[['Series', 'Court', 'Surface', 'Round', 'fav_coef', 'under_coef',
                 'fav_Rank', 'fav_Pts', 'fav_l1_year_pro', 'fav_l1_hand', 'fav_l1_height',
                 'under_Rank', 'under_Pts', 'under_l1_year_pro', 'under_l1_hand', 'under_l1_height',
                 'fav_1_win_rate','under_1_win_rate','fav_1_surf_win_rate', 'under_1_surf_win_rate'	,'fav_win']]

In [36]:
model_data = model_data.dropna().reset_index(drop=True)

выводим новые переменные, которые сравнивают разные характеристики игроков

In [37]:
model_data['Rank_diff'] = model_data['fav_Rank'] - model_data['under_Rank']
model_data['Pts_diff'] = model_data['fav_Pts'] - model_data['under_Pts']
model_data['Exp_diff'] = model_data['fav_l1_year_pro'] - model_data['under_l1_year_pro']
model_data['Same_hands'] = (model_data['fav_l1_hand']==model_data['under_l1_hand'])*1
model_data['Height_diff'] = model_data['fav_l1_height'] - model_data['under_l1_height']
model_data['win_diff'] = model_data['fav_1_win_rate'] - model_data['under_1_win_rate']
model_data['win_surf_diff'] = model_data['fav_1_surf_win_rate'] - model_data['under_1_surf_win_rate']

In [38]:
model_data = model_data.drop(columns=['fav_Rank', 'fav_Pts', 'fav_l1_year_pro', 'fav_l1_hand', 'fav_l1_height',
                 'under_Rank', 'under_Pts', 'under_l1_year_pro', 'under_l1_hand', 'under_l1_height',
                 'fav_1_win_rate','under_1_win_rate', 'fav_1_surf_win_rate', 'under_1_surf_win_rate'])

проводим one hot encoding для категориальных переменных и добавляем их в датасет

In [39]:
dummies = pd.get_dummies(model_data[['Series', 'Court', 'Surface', 'Round']], dtype=int)

In [40]:
model_data = model_data.drop(columns=['Series', 'Court', 'Surface', 'Round']).join(dummies)

определяем целевую переменную fav_win, которая нам показываем выиграли ли фаворит или нет

In [41]:
X = model_data.drop(columns=['fav_win'])
y = model_data['fav_win']

разбиваем на тестовую и тренировочную выборки

In [42]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=5)

Для модели я выбрал алгоритм LightGBM

In [43]:
model = ltb.LGBMClassifier(random_state=0, is_unbalance = True)

настраиваем параметры для подбора оптимальных параметров

In [44]:
parametrs = { 'boosting_type': ['gbdt', 'dart'],
             'max_depth': range(2,6),
             'n_estimators': range(40,161,20),
             'importance_type':['gain', 'split'],
             'num_leaves': [1,2,3,4]}

In [45]:
# grid = GridSearchCV(model, parametrs, cv=3, scoring='accuracy')
# grid.fit(X_train, y_train)
# grid.best_params_

задаем модель

In [46]:
model = ltb.LGBMClassifier(random_state=0, is_unbalance = True, boosting_type='gbdt', max_depth=3,
                           n_estimators=160, importance_type='gain', num_leaves=4)

In [47]:
model.fit(X_train,y_train)

LGBMClassifier(importance_type='gain', is_unbalance=True, max_depth=3,
               n_estimators=160, num_leaves=4, random_state=0)

In [48]:
test_predict = model.predict(X_test)
test_predict

array([1, 0, 0, ..., 1, 0, 1])

смотрим на матрицу ошибок

In [49]:
metrics.confusion_matrix(y_test, test_predict)

array([[1988,  588],
       [2352, 4039]])

смотрим на точность классификатора на тестовой выборке

In [50]:
metrics.accuracy_score(y_test, test_predict)

0.6721311475409836

проверяем не переобчена ли модель

In [51]:
model.score(X_train,y_train)

0.6815466233331741

модель не переобочена

In [52]:
test = X_test.join(y_test).reset_index(drop=True)
test['pred'] = test_predict

теперь посчитаем выигрыш при использовании предсказаний модели со стратегией флэта

In [53]:
a = []
for i in range(len(test)):
  if (test['fav_win'][i] == test['pred'][i]) and (test['fav_win'][i]==1):
    a.append(test['fav_coef'][i])
  elif (test['fav_win'][i] == test['pred'][i]) and (test['fav_win'][i]==0):
    a.append(test['under_coef'][i])
  else:
    a.append(0)    

In [54]:
print(f'На дистанции {len(a)} ставок выигрыш составляет {sum(a)-len(a)}, ROI составляет {((sum(a)-len(a))/len(a))*100}%')

На дистанции 8967 ставок выигрыш составляет 1121.6450000000787, ROI составляет 12.508587041374803%


Как можно заметить, при использовании классификатора результаты с точки зрения выигрыша заметно улучшились, относительно использования наивных предсказаний даже с учетом различных стратегий.